# Imports

In [ ]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch_optimizer as optim


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.width = 0
import warnings
import torchvision
warnings.filterwarnings('ignore')

from facebook_hateful_memes_detector.utils.globals import set_global, get_global
set_global("cache_dir", "/home/ahemf/cache/cache")

from facebook_hateful_memes_detector.utils import read_json_lines_into_df, in_notebook, set_device
get_global("cache_dir")
from facebook_hateful_memes_detector.models import Fasttext1DCNNModel, MultiImageMultiTextAttentionEarlyFusionModel
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, my_collate, get_datasets, get_image2torchvision_transforms, TextAugment
from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut, ImageAugment
from facebook_hateful_memes_detector.training import *
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)


In [ ]:


choice_probas = {"keyboard": 0.1, "char_substitute": 0.0, "char_insert": 0.1, "char_swap": 0.1, "ocr": 0.0, "char_delete": 0.1,
                 "fasttext": 0.0, "glove_twitter": 0.0, "glove_wiki": 0.0, "word2vec": 0.0, "split": 0.1,
                 "stopword_insert": 0.3, "word_join": 0.1, "word_cutout": 0.8,
                 "text_rotate": 0.5, "sentence_shuffle": 0.5, "one_third_cut": 0.3, "half_cut":0.1}
preprocess_text = TextAugment([0.05, 0.05, 0.05, 0.35, 0.3, 0.2], choice_probas, fasttext_file="wiki-news-300d-1M-subword.bin")

im_transform = transforms.RandomChoice([transforms.Grayscale(num_output_channels=3), 
                                        transforms.RandomHorizontalFlip(p=1.0),
                                        lambda x: x,
                                        transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)]),
                                        transforms.Compose([transforms.Resize(480), transforms.CenterCrop(400)]),
                                        transforms.Compose([transforms.Resize(640), transforms.CenterCrop(520)]),
                                        DefinedRotation(15), QuadrantCut()])

data = get_datasets(data_dir="../data/", train_text_transform=preprocess_text, train_image_transform=im_transform, 
                    test_text_transform=None, test_image_transform=None, 
                    cache_images = True, use_images = True, dev=False, 
                    keep_original_text=False, keep_original_image=False, 
                    keep_processed_image=True, keep_torchvision_image=False,)

# ImageAugment([0.2, 0.5, 0.3])


In [ ]:
sgd = torch.optim.SGD
sgd_params = dict(lr=2e-2,
                  momentum=0.9,
                  dampening=0,
                  weight_decay=0,
                  nesterov=False)

rangerQH = optim.RangerQH
rangerQHparams = dict(
    lr=1e-3,
    betas=(0.9, 0.999),
    nus=(.7, 1.0),
    weight_decay=0.0,
    k=6,
    alpha=.5,
    decouple_weight_decay=True,
    eps=1e-8,
)

adam = torch.optim.Adam
adam_params = params = dict(lr=1e-3, weight_decay=1e-6)

adamw = torch.optim.AdamW
adamw_params = dict(lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-2)

novograd = optim.NovoGrad
novograd_params = dict(
    lr=1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
    grad_averaging=False,
    amsgrad=False,
)

qhadam = optim.QHAdam
qhadam_params = dict(
    lr=1e-3,
    betas=(0.9, 0.999),
    nus=(1.0, 1.0),
    weight_decay=0,
    decouple_weight_decay=False,
    eps=1e-8,
)

radam = optim.RAdam
radam_params = dict(
    lr=1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
)

yogi = optim.Yogi
yogi_params = dict(lr=1e-2,
                   betas=(0.9, 0.999),
                   eps=1e-3,
                   initial_accumulator=1e-6,
                   weight_decay=0)

In [ ]:
batch_size=64
epochs = 5
optimizer = adam
optimizer_params = adam_params

scheduler_init_fn = get_multistep_lr([5, 7, 10, 17], gamma=0.1) # get_cosine_schedule_with_warmup # get_cosine_with_hard_restarts_schedule_with_warmup
scheduler_init_fn = get_cosine_schedule_with_warmup()
augmentation_weights = {"None": 1.0}




# Single Models

## LXMERT

In [ ]:
lr_strategy = {"lxmert": {"model":{"bert":{
    "embeddings": {"lr": optimizer_params["lr"]/1e4},
    "encoder": {"x_layers": {"lr": optimizer_params["lr"]/1e2}, "lr": optimizer_params["lr"]/1e3},
    "pooler": {"lr": optimizer_params["lr"]/1e1},
}}}}

In [ ]:


from facebook_hateful_memes_detector.models.MultiModal.VilBertVisualBert import VilBertVisualBertModel

model_fn = model_builder(VilBertVisualBertModel, 
                         dict(model_name=["lxmert"], num_classes=2,
                 gaussian_noise=0.25, dropout=0.15,
                              featurizer="transformer",final_layer_builder=fb_1d_loss_builder,
                 internal_dims=512, classifier_dims=256,
                 n_tokens_out=16, n_layers=2,task="classification", finetune=True),
                         per_param_opts_fn=lr_strategy,
                         optimiser_class=optimizer, optimiser_params=optimizer_params)


In [ ]:


n_tests = 1
multi_eval=False
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs


# 1h 45m


## Visual BERT / VilBert

In [ ]:
from facebook_hateful_memes_detector.models.MultiModal.VilBertVisualBert import VilBertVisualBertModel

model_fn = model_builder(VilBertVisualBertModel,
                         dict(
                             model_name=["vilbert"],
                             num_classes=2,
                             gaussian_noise=0.25,
                             dropout=0.15,
                             featurizer="transformer",
                             final_layer_builder=fb_1d_loss_builder,
                             internal_dims=512,
                             classifier_dims=256,
                             n_tokens_out=16,
                             n_layers=2,
                             task="classification",
                             finetune_vilbert=False,
                             finetune_lxmert=False,
                             finetune_visual_bert=False,
                         ),
                         per_param_opts_fn=None,
                         optimiser_class=optimizer,
                         optimiser_params=optimizer_params)

n_tests = 1
multi_eval = False
kfold = False
results, prfs = train_validate_ntimes(
    model_fn,
    data,
    batch_size,
    epochs,
    augmentation_weights=augmentation_weights,
    kfold=kfold,
    multi_eval=multi_eval,
    scheduler_init_fn=scheduler_init_fn,
    validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs

# 1h 45m

# Triple Bert Pooled

In [ ]:
batch_size = 32
lr_strategy = {
    "lxmert": {
        "lr": optimizer_params["lr"] / 100
    },
    "vilbert": {
        "lr": optimizer_params["lr"] / 100
    },
    "visual_bert": {
        "lr": optimizer_params["lr"] / 100
    },
}

from facebook_hateful_memes_detector.models.MultiModal.VilBertVisualBert import VilBertVisualBertModel

model_fn = model_builder(VilBertVisualBertModel,
                         dict(
                             model_name=["vilbert", "lxmert", "visual_bert"],
                             num_classes=2,
                             gaussian_noise=0.1,
                             dropout=0.35,
                             featurizer="pass",
                             final_layer_builder=fb_1d_loss_builder,
                             internal_dims=None,
                             classifier_dims=768,
                             n_tokens_out=64 + 100 + 64 + 36 + 64 + 100,
                             n_layers=2,
                             task="classification",
                             finetune_vilbert=False,
                             finetune_lxmert=True,
                             finetune_visual_bert=False,
                         ),
                         per_param_opts_fn=None,
                         optimiser_class=optimizer,
                         optimiser_params=optimizer_params)

n_tests = 1
multi_eval = False
kfold = False
results, prfs = train_validate_ntimes(
    model_fn,
    data,
    batch_size,
    epochs,
    augmentation_weights=augmentation_weights,
    kfold=kfold,
    multi_eval=multi_eval,
    scheduler_init_fn=scheduler_init_fn,
    validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs

# 1h 45m

# Triple Bert Transformer

In [ ]:
batch_size=32
from facebook_hateful_memes_detector.models.MultiModal.VilBertVisualBert import VilBertVisualBertModel

model_fn = model_builder(VilBertVisualBertModel, 
                         dict(model_name=["vilbert", "lxmert", "visual_bert"], num_classes=2,
                 gaussian_noise=0.25, dropout=0.15,
                              featurizer="transformer",final_layer_builder=fb_1d_loss_builder,
                 internal_dims=256, classifier_dims=256,
                 n_tokens_out=32, n_layers=2,task="classification", finetune=True),
                         per_param_opts_fn=None,
                         optimiser_class=optimizer, optimiser_params=optimizer_params)




n_tests = 1
multi_eval=False
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs


# 1h 45m



# Predict

In [ ]:
batch_size=16
epochs = 5

submission, text_model = train_and_predict(model_fn, data, batch_size, epochs, augmentation_weights, scheduler_init_fn=scheduler_init_fn)
submission.to_csv("submission.csv",index=False)
submission.sample(3)
